# Reformer with Visual Tokens

In [1]:
import argparse
import os
import sys
import numpy as np
import math
import time
import pickle
import cv2 as cv
import matplotlib
import matplotlib.pyplot as plt
import random
from cv2 import VideoWriter, VideoWriter_fourcc, imread

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler

import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision

import warnings

from ModelZoo import TemporalDiscriminator, StyleGanGenerator, StyleGanDiscriminator
import transformers
from transformers import AutoTokenizer, AutoModelWithLMHead

import h5py
import tensorflow as tf

BATCH_SIZE = 48

In [2]:
# tokenizer = AutoTokenizer.from_pretrained("google/reformer-enwik8")
model = transformers.ReformerModelWithLMHead.from_pretrained("google/reformer-enwik8").to(torch.device('cuda'))

C:\ProgramData\Anaconda3\envs\pd\lib\site-packages\torch\nn\modules\container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [3]:
def encode(list_of_strings, pad_token_id=0):
    max_length = max([len(string) for string in list_of_strings])

    # create emtpy tensors
    attention_masks = torch.zeros((len(list_of_strings), max_length), dtype=torch.long)
    input_ids = torch.full((len(list_of_strings), max_length), pad_token_id, dtype=torch.long)

    for idx, string in enumerate(list_of_strings):
        # make sure string is in byte format
        if not isinstance(string, bytes):
            string = str.encode(string)

        input_ids[idx, :len(string)] = torch.tensor([x + 2 for x in string])
        attention_masks[idx, :len(string)] = 1

    return input_ids, attention_masks

def decode(outputs_ids):
    decoded_outputs = []
    for output_ids in outputs_ids.tolist():
        # transform id back to char IDs < 2 are simply transformed to ""
        decoded_outputs.append("".join([chr(x - 2) if x > 1 else "" for x in output_ids]))
    return decoded_outputs

SyntaxError: invalid syntax (<ipython-input-3-069a8ea6a767>, line 17)

In [ ]:
encoded, attention_masks = encode(["First we will see how to easily leverage the pipeline API to quickly use those pretrained models at inference. Then, we will dig a little bit more and see how the library gives you access to those models and helps you preprocess your data."])

In [ ]:
print(decode(model.generate(encoded.to(torch.device('cuda')), do_sample=True, max_length=10000)))

In [20]:
m = [i for i in model.children()]
m = [i for i in m[0].children()]
m = m[1]
print(m)

ReformerEncoder(
  (layers): ModuleList(
    (0): ReformerLayer(
      (attention): ReformerAttention(
        (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (self_attention): LocalSelfAttention(
          (query): Linear(in_features=1024, out_features=1024, bias=False)
          (key): Linear(in_features=1024, out_features=1024, bias=False)
          (value): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (output): ReformerSelfOutput(
          (dense): Linear(in_features=1024, out_features=1024, bias=False)
        )
      )
      (feed_forward): ChunkReformerFeedForward(
        (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dense): ReformerFeedForwardDense(
          (dense): Linear(in_features=1024, out_features=4096, bias=True)
        )
        (output): ReformerFeedForwardOutput(
          (dense): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (1): Reform